<a href="https://colab.research.google.com/github/jaswanthreddy049/GEN_AI/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install llama-index sentence-transformers chromadb gradio llama-index-llms-groq llama-index-vector-stores-chroma llama-index-embeddings-huggingface langchain langchain-cohere cohere llama-index-postprocessor-cohere-rerank

In [55]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.groq import Groq
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from llama_index.core.query_engine import RetrieverQueryEngine

In [56]:
from google.colab import files
uploaded = files.upload()

Saving HR Policy Manual 2023.pdf to HR Policy Manual 2023 (1).pdf


In [40]:
filename = list(uploaded.keys())[0]
reader = PDFReader()
documents = reader.load_data(filename)
#use unstructured later

In [88]:

parser = SentenceSplitter(
    chunk_size=750,
    chunk_overlap=150
)
nodes = parser.get_nodes_from_documents(documents)

In [89]:
# Load the BGE-Large v1.5 model
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

# Generate embeddings for the split documents
# Assuming split_documents is a list of LlamaIndex TextNode objects
embeddings_list = embedding_model.get_text_embedding_batch([node.text for node in nodes])

In [168]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.get_or_create_collection("my_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex(
    nodes,
    vector_store=vector_store,
    embed_model=embedding_model  # e.g. BGE-large
)

In [170]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('demo')


llm = Groq(
    model="llama3-70b-8192",
    temperature=0.3,
)


In [171]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20 # number of chunks to retrieve
)

In [172]:
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    llm=llm
)
#[tree_summarize]
#Breaks k retrieved chunks into small groups (like a tree's branches)
#Uses the LLM (llm=llm) to summarize each group
#Combines those summaries recursively
#Generates a single coherent final summary/answer

In [173]:
from llama_index.postprocessor.cohere_rerank import CohereRerank
from google.colab import userdata
import os

# Set the Cohere API key as an environment variable
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')
#reranker = CohereRerank(model="rerank-english-v3.0") # This line might already be in your Step 2

# Instantiate the LlamaIndex CohereRerank postprocessor
# The `top_n` parameter specifies how many top documents to keep *after* reranking
cohere_reranker_li = CohereRerank(
    api_key=os.environ["COHERE_API_KEY"], # Use the API key from your environment variable
    model="rerank-english-v3.0",
    top_n=3 # Keep the top 3 documents after reranking. Adjust as needed.
)

# Create the QueryEngine, adding the CohereRerank postprocessor
# We will explicitly create RetrieverQueryEngine later using the custom retriever and this postprocessor
# This cell should now only define the CohereRerank postprocessor

# query_engine = index.as_query_engine(
#     similarity_top_k=10, # IMPORTANT: Retrieve MORE documents initially (e.g., 10-20)
#                           # so the reranker has a good pool to work with.
#                           # The reranker will then narrow it down to `top_n` (e.g., 3).
#     node_postprocessors=[cohere_reranker_li], # Add the reranker here
#     llm=llm # Explicitly pass the Groq LLM
# )

In [174]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[cohere_reranker_li],

)

In [138]:
query = "What is the difference between aerobic and anaerobic respiration?"
response = query_engine.query(query)
print(response)

I apologize, but the provided context information does not seem to be related to the query about aerobic and anaerobic respiration. The context appears to be related to a HR Policy Manual, travel policies, and forms for travel reimbursement, whereas the query is about a biological concept.

To answer the query, I would need to provide information that is not present in the context. Aerobic and anaerobic respiration are two types of cellular respiration, which are biological processes that occur in cells to generate energy. Aerobic respiration occurs in the presence of oxygen, whereas anaerobic respiration occurs in the absence of oxygen. Aerobic respiration produces more energy and is characteristic of most eukaryotic cells, while anaerobic respiration produces less energy and is characteristic of some prokaryotic cells and muscle cells during high-intensity exercise.

Please note that this answer is not based on the provided context information, but rather on general knowledge about b

In [51]:
import gradio as gr

def ask_bot(query):
    return query_engine.query(query)

gr.Interface(fn=ask_bot, inputs="text", outputs="text", title="EduQuery").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87e838753b0e7d29e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [175]:
evaluation_data=[
  {
    "query": "What is the objective of the IIMA HR Policy Manual?",
    "ground_truth": "The objective of the IIMA HR Policy Manual is to compile the HR policies and procedures followed at IIMA and present the general rules and regulations that govern the employees of the Institute."
  },
  {
    "query": "What types of leave are available to IIMA employees?",
    "ground_truth": "IIMA employees are entitled to Casual Leave, Earned Leave, Half Pay Leave, Commuted Leave, Extraordinary Leave, Maternity Leave, Paternity Leave, and Leave for adoption of a child by female employees."
  },
  {
    "query": "What is the recruitment process for manager-level posts at IIMA?",
    "ground_truth": "For manager-level posts, the recruitment process includes submitting a Manpower Requisition Form, advertising the position, processing applications, conducting interviews with a panel, and final selection upon medical clearance."
  },
  {
    "query": "What is the role of the CMGI in handling sexual harassment cases?",
    "ground_truth": "CMGI is responsible for investigating complaints of sexual harassment, conducting inquiries, providing redressal, maintaining confidentiality, and ensuring protection against victimization."
  },

  {
    "query": "What are the working hours defined for employees?",
    "ground_truth": "The standard office hours for employees are from 9:00 AM to 5:30 PM, with variations depending on roles. Attendance is mandatory during working hours."
  }
]


In [176]:
results = []

for item in evaluation_data:
    query = item["query"]
    response = query_engine.query(query)  # or retriever.query() based on your setup
    results.append({
        "query": query,
        "ground_truth": item["ground_truth"],
        "generated_answer": str(response)
    })


In [59]:
!pip install rouge-score


In [177]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

for r in results:
    score = scorer.score(r["ground_truth"], r["generated_answer"])
    r["rougeL"] = score["rougeL"].fmeasure


In [178]:
# During retrieval
retrieved_ids = retriever.retrieve(query)  # list of top 5 ids

# If you know the correct doc ID that contains the ground truth
ground_truth_id = query_engine.query(query)  # Define this per your setup

recall_at_5 = int(ground_truth_id in retrieved_ids)


In [179]:
retrieved_texts = [node.text for node in retriever.retrieve(query)]

recall_at_5 = int(any(item["ground_truth"] in doc for doc in retrieved_texts))


In [180]:
for r in results:
    print("Query:", r["query"])
    print("Ground Truth:", r["ground_truth"])
    print("Generated:", r["generated_answer"])
    print("ROUGE-L:", r["rougeL"])
    print("Recall@5:", r.get("recall@5", "N/A"))
    print("="*50)
print("Average ROUGE-L:", sum(r["rougeL"] for r in results) / len(results))


Query: What is the objective of the IIMA HR Policy Manual?
Ground Truth: The objective of the IIMA HR Policy Manual is to compile the HR policies and procedures followed at IIMA and present the general rules and regulations that govern the employees of the Institute.
Generated: The objective of the IIMA HR Policy Manual is to compile the HR policies and procedures followed in IIMA, and to present the general rules and regulations that govern the employees of the Institute.
ROUGE-L: 0.955223880597015
Recall@5: N/A
Query: What types of leave are available to IIMA employees?
Ground Truth: IIMA employees are entitled to Casual Leave, Earned Leave, Half Pay Leave, Commuted Leave, Extraordinary Leave, Maternity Leave, Paternity Leave, and Leave for adoption of a child by female employees.
Generated: There are 7 types of leave available to IIMA employees: Commuted Leave, Half Pay Leave, Extraordinary Leave, Maternity Leave, Paternity Leave, and two unspecified types of leave (Leave Type 1 and